In [25]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import pipeline

In [2]:
books = pd.read_csv("books_with_categories.csv")

In [6]:
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k=None,
                      device="cuda")
classifier("I love this!")

[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528691716492176},
  {'label': 'neutral', 'score': 0.005764602683484554},
  {'label': 'anger', 'score': 0.004419790115207434},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.0016119946958497167},
  {'label': 'fear', 'score': 0.0004138525982853025}]]

In [7]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [8]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548407673835754},
  {'label': 'neutral', 'score': 0.16985225677490234},
  {'label': 'sadness', 'score': 0.11640913784503937},
  {'label': 'surprise', 'score': 0.02070065215229988},
  {'label': 'disgust', 'score': 0.019100721925497055},
  {'label': 'joy', 'score': 0.015161371789872646},
  {'label': 'anger', 'score': 0.003935147542506456}]]

### Since some descriptions can capture different kinds of emotions, it is not viable that we use/classify on the whole description as we could miss some informations
- What we can do is split the description with each individual sentences and classify on that basis

In [9]:
# Luckily the model allows us to capture sentiment analysis on more than 1 sentences at the same time
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296029925346375},
  {'label': 'neutral', 'score': 0.14038561284542084},
  {'label': 'fear', 'score': 0.06816210597753525},
  {'label': 'joy', 'score': 0.04794234782457352},
  {'label': 'anger', 'score': 0.009156347252428532},
  {'label': 'disgust', 'score': 0.0026284726336598396},
  {'label': 'sadness', 'score': 0.0021221598144620657}],
 [{'label': 'neutral', 'score': 0.44937095046043396},
  {'label': 'disgust', 'score': 0.2735915184020996},
  {'label': 'joy', 'score': 0.10908295959234238},
  {'label': 'sadness', 'score': 0.09362708777189255},
  {'label': 'anger', 'score': 0.040478259325027466},
  {'label': 'surprise', 'score': 0.026970149949193},
  {'label': 'fear', 'score': 0.006879040505737066}],
 [{'label': 'neutral', 'score': 0.6462162733078003},
  {'label': 'sadness', 'score': 0.2427331507205963},
  {'label': 'disgust', 'score': 0.04342266544699669},
  {'label': 'surprise', 'score': 0.02830052375793457},
  {'label': 'joy', 'score': 0.0142114469

In [10]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

### Analyzing sentiment analysis results

In [11]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [12]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296029925346375},
 {'label': 'neutral', 'score': 0.14038561284542084},
 {'label': 'fear', 'score': 0.06816210597753525},
 {'label': 'joy', 'score': 0.04794234782457352},
 {'label': 'anger', 'score': 0.009156347252428532},
 {'label': 'disgust', 'score': 0.0026284726336598396},
 {'label': 'sadness', 'score': 0.0021221598144620657}]

In [13]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [14]:
predictions[3]

[{'label': 'fear', 'score': 0.928168535232544},
 {'label': 'anger', 'score': 0.032190799713134766},
 {'label': 'neutral', 'score': 0.012808662839233875},
 {'label': 'sadness', 'score': 0.008756822906434536},
 {'label': 'surprise', 'score': 0.008597886189818382},
 {'label': 'disgust', 'score': 0.008431801572442055},
 {'label': 'joy', 'score': 0.0010455806041136384}]

In [15]:
# Sort by labels instead of scores, so that we have uniformity across all predictions
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156347252428532},
 {'label': 'disgust', 'score': 0.0026284726336598396},
 {'label': 'fear', 'score': 0.06816210597753525},
 {'label': 'joy', 'score': 0.04794234782457352},
 {'label': 'neutral', 'score': 0.14038561284542084},
 {'label': 'sadness', 'score': 0.0021221598144620657},
 {'label': 'surprise', 'score': 0.7296029925346375}]

In [22]:
# This will be the columns later on in our dataset, and will be the keys to our dictionary
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]

# Empty list to store isbn13 which are our identifiers
isbn = []

# This will contain all of the scores for every single description for each of our labels
emotion_scores = {label : [] for label in emotion_labels}

# function to calculate max emotion scores
def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label : [] for label in emotion_labels} # Hold all the predictions for a particular description

    # Loop over each sentence, returned per prediction
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"]) # This ensures that all labels/emotions are in the same order

        # Loop over the sorted predictions
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"]) # Extract all emotion scores, and append it using the correct label

    return {label : np.max(scores) for label, scores in per_emotion_scores.items()} # return the maximum for each emotion/label

In [23]:
# testing the function in 10 entries
for i in range(10):
    isbn.append(books["isbn13"][i]) # append isbn for the current entry
    sentences = books["description"][i].split(".") # Split current entry into individual sentences
    predictions = classifier(sentences) # Predict emotion scores using classifier
    max_scores = calculate_max_emotion_scores(predictions) # Calculate max scores for each label

    # Store nax labels in emotion_scores list
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [24]:
emotion_scores

{'anger': [0.06413359194993973,
  0.6126184463500977,
  0.06413359194993973,
  0.35148340463638306,
  0.08141231536865234,
  0.232224702835083,
  0.5381850600242615,
  0.06413359194993973,
  0.3006700575351715,
  0.06413359194993973],
 'disgust': [0.2735915184020996,
  0.3482851982116699,
  0.10400661081075668,
  0.1507224589586258,
  0.18449534475803375,
  0.7271749377250671,
  0.1558549404144287,
  0.10400661081075668,
  0.2794812321662903,
  0.17792819440364838],
 'fear': [0.928168535232544,
  0.9425278306007385,
  0.9723208546638489,
  0.36070704460144043,
  0.09504338353872299,
  0.051362790167331696,
  0.747428297996521,
  0.40449538826942444,
  0.9155242443084717,
  0.051362790167331696],
 'joy': [0.9327974319458008,
  0.7044215202331543,
  0.7672373056411743,
  0.25188103318214417,
  0.040564391762018204,
  0.04337579384446144,
  0.8725650906562805,
  0.040564391762018204,
  0.040564391762018204,
  0.040564391762018204],
 'sadness': [0.6462162733078003,
  0.8879395127296448,
  

In [26]:
# Resetting our variables to start from scratch
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label : [] for label in emotion_labels}

# Loop over all book description entries
for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i]) # append isbn for the current entry
    sentences = books["description"][i].split(".") # Split current entry into individual sentences
    predictions = classifier(sentences) # Predict emotion scores using classifier
    max_scores = calculate_max_emotion_scores(predictions) # Calculate max scores for each label

    # Store nax labels in emotion_scores list
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [02:00<00:00, 42.96it/s]


In [27]:
# Save results to dataframe
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [28]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273592,0.928169,0.932797,0.646216,0.967158,0.729603,9780002005883
1,0.612618,0.348285,0.942528,0.704422,0.887940,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,9780006178736
3,0.351483,0.150722,0.360707,0.251881,0.732685,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148208,0.030643,0.919165,0.255171,0.853722,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459269,0.759455,0.951104,0.368111,0.078765,9788185300535


In [29]:
# Merge results to our dataframe
books = pd.merge(books, emotions_df, on="isbn13")

In [30]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273592,0.928169,0.932797,0.646216,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612618,0.348285,0.942528,0.704422,0.887940,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.150722,0.360707,0.251881,0.732685,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148208,0.030643,0.919165,0.255171,0.853722,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459269,0.759455,0.951104,0.368111,0.078765


In [31]:
books.to_csv("books_with_emotions.csv", index=False)